## Code Completion

<a target="_blank" href="https://colab.research.google.com/github/microsoft/LLMLingua/blob/main/examples/Code.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In addition to natural language, we also evaluate the capabilities of our method on code. For example, in [**repobench-p**](https://arxiv.org/abs/2306.03091), the original prompts can be particularly long, up to **20k** characters. LongLLMLingua can achieve a 1.4 point improvement at a **6x** compression ratio.

### RepoBench-p

Next, we will demonstrate the use of LongLLMLingua on the **RepoBench-P** dataset (Code Completion Task). This dataset contains samples of incomplete code snippets and several related reference code files. The task is to supplement and complete the next line of code.  
   
The original dataset can be found at https://huggingface.co/datasets/THUDM/LongBench.

In [1]:
# Install dependency.
!pip install llmlingua datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [2]:
# Download the original prompt and dataset
from datasets import load_dataset

dataset = load_dataset("THUDM/LongBench", "repobench-p", split="test")

In [8]:
# Using the OAI
import openai

openai.api_key = "<insert_openai_key>"

In [14]:
# or Using the AOAI
import openai

openai.api_key = "<insert_openai_key>"
openai.api_base = "https://xxxx.openai.azure.com/"
openai.api_type = "azure"
openai.api_version = "2023-05-15"

### Setup Data

In [12]:
# select an example from MeetingBank
contexts, question, answer = [
    dataset[1][key] for key in ["context", "input", "answers"]
]
instruction = "Please complete the code given below."
question = question + "\n\nNext line of code:\n"

In [13]:
# Ground-truth Answer
answer

['        mEffects.add(new NoEffect());']

### The response of Original prompt

In [16]:
# The response from original prompt, using GPT-4-32k
import json

prompt = "\n\n".join([instruction, contexts, question])

message = [
    {"role": "user", "content": prompt},
]

request_data = {
    "messages": message,
    "max_tokens": 100,
    "temperature": 0,
    "top_p": 1,
    "n": 1,
    "stream": False,
}
response = openai.ChatCompletion.create(
    "gpt-4-32k",
    **request_data,
)
print(json.dumps(response, indent=4))

{
    "id": "chatcmpl-8FjSIsMsihREbjFpWGGSHgT6OpNqK",
    "object": "chat.completion",
    "created": 1698760342,
    "model": "gpt-4-32k",
    "choices": [
        {
            "index": 0,
            "finish_reason": "stop",
            "message": {
                "role": "assistant",
                "content": "The next line of code is missing. Please provide more information."
            }
        }
    ],
    "usage": {
        "prompt_tokens": 18817,
        "completion_tokens": 13,
        "total_tokens": 18830
    }
}


### The response of Compressed Prompt (Correct in 10x Compression)

In [18]:
# Setup LLMLingua
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/hjiang/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/hjiang/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [29]:
contexts_list = contexts.split("\n")
contexts_list = [
    "\n".join(contexts_list[ii : ii + 4]) for ii in range(0, len(contexts_list), 4)
]

In [30]:
# 8x Compression
compressed_prompt = llm_lingua.compress_prompt(
    contexts_list,
    instruction=instruction,
    question=question,
    target_token=2000,
    condition_compare=True,
    condition_in_question="after",
    rank_method="longllmlingua",
    use_sentence_level_filter=False,
    context_budget="+100",
    dynamic_context_compression_ratio=0.4,  # enable dynamic_context_compression_ratio
    reorder_context="sort",
)
message = [
    {"role": "user", "content": compressed_prompt["compressed_prompt"]},
]

request_data = {
    "messages": message,
    "max_tokens": 100,
    "temperature": 0,
    "top_p": 1,
    "n": 1,
    "stream": False,
}
response = openai.ChatCompletion.create(
    "gpt-4-32k",
    **request_data,
)

print(json.dumps(compressed_prompt, indent=4))
print("Response:", response)

{
    "compressed_prompt": "Please complete the code given below.\n\n   };\n}\nSpectaculum-Demo/src/main/java/net/protyposis/android/spectaculumdemo/testeffect/InterlaceEffect.java\npublic class InterlaceEffect ShaderEffect {\n\n        return new TextureKernelShaderProgram(TextureKernelShaderProgram.Kernel.BLUR_GAUSS);\n    }\n}\nSpectaculum-Effect-FlowAbs/src/main/java/net/protyposis/android/spectaculum/effects/FlowAbsEffect.java\n\n    void removeListener(Listener listener);\n}\nSpectaculum-Effect-Immersive/src/main/java/net/protyposis/android/spectaculum/effects/ImmersiveSensorNavigation.java\npublic class ImmersiveSensorNavigation implements SensorEventListener {\n\n    }\n}\nSpectaculum-Effect-FlowAbs/src/main/java/net/protyposis/android/spectaculum/effects/FlowAbsSubEffect.java\npublic abstract class FlowAbsSubEffect extends BaseEffect {\n\n    }\n}\nSpectaculum-Core/src/main/java/net/protyposis/android/spectaculum/effects/ContrastBrightnessAdjustmentEffect.java\npublic class Co